ML Final Project

Spring 2017 - NYU CUSP

Notebook reads in datetime converted CitiBike Data by month (e.g. Jan_citi.csv) from 2016 and converts gets the counts and means per month

Written in Python 2.7 (Anaconda)

In [1]:
import numpy as np
import pandas as pd
#import geopandas as gp
#from geopandas import GeoDataFrame
import os
import io
import subprocess
from shapely.geometry import Point
from datetime import datetime
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from datetime import datetime
import matplotlib.pyplot as plt
import sklearn
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score as rs
from sklearn import svm, datasets
from sklearn import metrics
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import IsolationForest

from itertools import compress

np.random.seed(222)

%matplotlib inline
%pylab inline
pd.options.display.max_columns = 120
pd.options.display.max_rows = 120

Populating the interactive namespace from numpy and matplotlib


/Users/christopherstreich/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/christopherstreich/anaconda2/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['compress', 'datetime']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
proj_folder = os.getcwd()

Check for spurious entires

## Remove spurious trip length rationale

From website: A $101 security deposit hold will be placed on your card when you purchase a pass. If you incur usage

fees, your card will be charged. The fee for a lost or stolen bike is $1200 (+ tax).


## Total Loss: 

Annual pass = 163/year + 101 Security fee + ($ 2.50 * min) = 5616 minutes
 
3 day Pass = 24 + 101 Security fee + ($ 2.50 * 15min) = 4031.25

Day Pass = 12 + 101 Security fee + ($4 * 15min) = 4076.25

Ride = 4 + 101 Security fee + ($4 * 15min) = 4106.25 

So remove anything longer than 5616 minutes, the maximum time before Total Loss and keep trip long than 1 minute

In [3]:

def citi_agg_stat_day(label):
    '''This function is supposed to: Further clean citi bike data, Group by day, 
    get daily count and mean and then export the files to prepare for a large merge'''
    #Import the csv
    getfile = "data/" + label + ".csv"
    citi_agg = pd.read_csv(getfile)
    #Drop column
    citi_agg.drop('Unnamed: 0', axis=1, inplace=True)
    #change seconds to minutes
    citi_agg['tripduration'] = citi_agg['tripduration'].apply(lambda x: x / 60)
    #Remove spurious trips
    citi_agg = citi_agg[(citi_agg.tripduration > 1) & (citi_agg.tripduration < 5616)]
    #DatTime again??
    citi_agg['starttime'] = pd.to_datetime(citi_agg['starttime'])
    citi_agg['stoptime'] = pd.to_datetime(citi_agg['stoptime'])
    #Count Trips per day 
    citi_agg['bike_date'] = citi_agg['starttime'].dt.date
    citi_day_count = citi_agg.groupby(['bike_date'], as_index=False)['tripduration'].count()
    citi_day_count.columns = ['bike_date', 'bike_trip_count']
    
    #take the mean of the taxi stats each day
    citi_stat_agg = citi_agg.groupby(['bike_date'], as_index=False).mean()
    
    #Merge the seperate DataFrames 
    citi_df_by_day = citi_stat_agg.merge(citi_day_count, on='bike_date')
    
    #Get the weekday
    citi_df_by_day['weekday'] = 0
    for i in range(len(citi_df_by_day)):
        citi_df_by_day['weekday'].iloc[i] = citi_df_by_day.bike_date.iloc[i].weekday()
        
    #Export to csv
    output = "data/" + label + "_df_by_day.csv"
    citi_df_by_day.to_csv(output)
    #citi_df_byday_all
    return citi_df_by_day 

In [62]:
def citi_daystation_count(label):
    '''This function is supposed to: Further clean citi bike data, do some grouping
    and calculations and then export the files to prepare for a large merge'''
    #Import the csv
    getfile1 = "data/" + label + ".csv"
    citi_agg1 = pd.read_csv(getfile1)
    #Drop column
    citi_agg1.drop('Unnamed: 0', axis=1, inplace=True)
    #change seconds to minutes
    citi_agg1['tripduration'] = citi_agg1['tripduration'].apply(lambda x: x / 60)
    #Remove spurious trips
    citi_agg1 = citi_agg1[(citi_agg1.tripduration > 1) & (citi_agg1.tripduration < 5616)]
    #DatTime again??
    citi_agg1['starttime'] = pd.to_datetime(citi_agg1['starttime'])
    citi_agg1['stoptime'] = pd.to_datetime(citi_agg1['stoptime'])
    #citi_agg['day_number'] = label.starttime.apply(lambda x: x.hour)
    citi_agg1['bike_date'] = citi_agg1['starttime'].dt.date
    
    
    #Count Trips per day 
    citi_agg1['bike_date'] = citi_agg1['starttime'].dt.date
    citi_day_count1 = citi_agg1.groupby(['bike_date', "start_station id"], as_index=False)['tripduration'].count()
    citi_day_count1.columns = ['bike_date', "start_station id",'bike_trip_count']
    return citi_day_count1 

## Aggregate by day and Export

In [6]:
columns =['bike_date', 'tripduration', 'start_station id', 'start_station_latitude', 'start_station_longitude', \
          'end_station_id', 'end_station_latitude','end_station_longitude', 'bikeid', 'birth_year', \
          'gender', 'start_hour', 'end_hour','bike_trip_count', 'weekday']

In [8]:
months = ["Jan_citi", "Feb_citi", "Mar_citi", "Apr_citi", "May_citi", "Jun_citi", "Jul_citi", "Aug_citi", \
          "Sep_citi", "Oct_citi", "Nov_citi", "Dec_citi"]

In [27]:
#Create Empty dataframe
citi_df_all_day = pd.read_csv(io.StringIO(None), names=columns, dtype=dict(zip(columns,[object, float, float, float, float, float, float, float, float, float, float, float, float, int, int])))

In [10]:
#Create the files in bulk
By day with count and mean
for i in (months):
    x = citi_agg_stat_day(str(i))
    citi_df_all_day = citi_df_all_day.append(x, ignore_index = True)
        

/Users/christopherstreich/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [14]:
citi_df_all_day.to_csv('data/citi_df_all_day.csv')

In [80]:
citi_df_all_day.head()

,bike_date,tripduration,start_station id,start_station_latitude,start_station_longitude,end_station_id,end_station_latitude,end_station_longitude,bikeid,birth_year,gender,start_hour,end_hour,bike_trip_count,weekday


## Grouping by station and time of day

In [69]:
months = ["Jan_citi", "Feb_citi", "Mar_citi", "Apr_citi", "May_citi", "Jun_citi", "Jul_citi", "Aug_citi", \
          "Sep_citi", "Oct_citi", "Nov_citi", "Dec_citi"]

In [70]:
columns2 =['bike_date', 'start_station id', 'bike_trip_count']

In [71]:
citi_day_by_station = pd.read_csv(io.StringIO(None), names=columns2, dtype=dict(zip(columns,[object, int, int])))

In [72]:
#Get Station count grouped by day
for i in (months):
    y = citi_daystation_count(str(i))
    citi_day_by_station = citi_day_by_station.append(y, ignore_index = True)

In [77]:
citi_day_by_station.to_csv('data/citi_day_by_station.csv')

## Clustering Anomaly Detection

### Gaussian Mixture Model Anomaly Detection

In [ ]:
X = tlc_df_by_day[['bike_trip_count', 'tripduration']]
X.head(2)

In [79]:
citi_df_all_day.head(3)

,bike_date,tripduration,start_station id,start_station_latitude,start_station_longitude,end_station_id,end_station_latitude,end_station_longitude,bikeid,birth_year,gender,start_hour,end_hour,bike_trip_count,weekday


In [88]:
import pyshp as pys

ImportError: No module named pyshp

In [ ]:
install_name_tool -id libspecial.dylib libspecial.dylib

In [ ]:
libSystem.B.dylib

In [ ]:
Run: Christophers-MacBook-Pro:lib christopherstreich$ otool -L /Users/christopherstreich/anaconda2/lib/libgdal.20.dylib
        

In [ ]:
Get: @rpath/libjpeg.8.dylib (compatibility version 13.0.0, current version 13.0.0)